# Transfer Learning EfficientV2B3 Raw Data i224 b32 e30 ft2e20

## Details 

In this notebook the EfficientV2B3 model will be trained on the Raw Dataset. 

Image size is 224, Batch size is 32, Epochs is 30, model is trained then finetuned again on the same dataset.


### Importing Libraries

In [1]:
import os
import math
import json
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

2025-01-31 22:53:52.578701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738364032.701332   32905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738364032.735534   32905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D,
    MaxPool2D,
    Flatten,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
    BatchNormalization,
    Activation,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3, EfficientNetV2B3
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from sklearn.utils.class_weight import compute_class_weight
from datetime import datetime
from pathlib import Path

In [4]:
# Global Variables
MODEL_NAME = "tl_efficientv2b3_raw_i224_b32_e30_ft2e20"

NUM_CLASSES = 72
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30

# Get the base directory
BASE_DIR = Path.cwd().parent.parent

now = datetime.now()

### Importing Dataset

#### Dataset Preparation

In [5]:
# Define paths
DATA_DIR = BASE_DIR / "datasets" / "plant_disease_dataset_noaugmentation_raw"
SAMPLE_IMAGE = (
    BASE_DIR / "datasets" / "cropped_plant_village_dataset" / "sample_image.JPG"
)

print("Directory:", DATA_DIR)
print("Sample Image Path:", SAMPLE_IMAGE)

Directory: /home/sam5io/sam_engineerings/AgroDiagnoseAI/datasets/plant_disease_dataset_noaugmentation_raw
Sample Image Path: /home/sam5io/sam_engineerings/AgroDiagnoseAI/datasets/cropped_plant_village_dataset/sample_image.JPG


##### Data split

In [6]:
# Load training dataset
training_set = image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

# Load validation dataset
validation_set = image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 116232 files belonging to 72 classes.
Using 92986 files for training.


I0000 00:00:1738364419.566969   32905 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


KeyboardInterrupt: 

In [ ]:
# Load test dataset
test_set = image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

#### Dataset Details

In [ ]:
# import numpy as np
from collections import Counter

# Get class names
class_names = training_set.class_names

# Initialize a counter for each class
class_counts = Counter()

for images, labels in training_set:
    label_indices = np.argmax(
        labels.numpy(), axis=1
    )  # Convert one-hot to class indices
    class_counts.update(label_indices)

# Print class-wise image count
for class_idx, count in class_counts.items():
    print(f"Class '{class_names[class_idx]}': {count} images")

In [33]:
# # # Function to count images in each class
# # def count_images_in_classes(dataset_dir):
# #     """
# #     Counts the number of images in each class within a dataset directory.

# #     Args:
# #         dataset_dir (str): The path to the dataset directory.

# #     Returns:
# #         dict: A dictionary where the keys are the class names and the values are the number of images in each class.
# #     """
# #     class_counts = {}
# #     for class_name in os.listdir(dataset_dir):
# #         class_path = os.path.join(dataset_dir, class_name)
# #         if os.path.isdir(class_path):
# #             class_counts[class_name] = len(os.listdir(class_path))
# #     return class_counts


# # # Count images in training and validation sets
# # train_class_counts = count_images_in_classes(TRAIN_DIR)
# # valid_class_counts = count_images_in_classes(VALID_DIR)

# # Create a DataFrame for better visualization
# df = pd.DataFrame(
#     {
#         "Class": list(training_set.class_names),
#         "Training Images": list(training_set),
#         "Validation Images": 
#             list(training_set)
#         # [
#             # valid_class_counts.get(cls, 0) for cls in train_class_counts.keys()
#         # ],
#     }
# ).sort_values(by="Class", ascending=True)

In [ ]:
# # Display the DataFrame
# print(df)

In [ ]:
# # Plot the class distribution with adjustments for readability
# df.plot(
#     x="Class", kind="bar", stacked=True, figsize=(20, 8), title="Class Distribution"
# )
# plt.ylabel("Number of Images")
# plt.xlabel("Class")

# # Rotate x-ticks for better readability
# plt.xticks(rotation=90, ha="center")

# # Adjust layout to prevent clipping of labels
# plt.tight_layout()

# # Show the plot
# plt.show()

In [36]:
# # Function to visualize one image per class
# def visualize_sample_images(dataset_dir):
#     """
#     Visualizes one sample image per class in the dataset directory.

#     Args:
#         dataset_dir (str): The path to the dataset directory.

#     Displays a grid of images, with one image per class, using matplotlib.
#     """
#     class_names = os.listdir(dataset_dir)
#     class_names.sort()  # Sort for consistent order

#     # Calculate the number of rows and columns for the subplot grid
#     num_classes = len(class_names)
#     num_cols = 5  # You can adjust this number
#     num_rows = math.ceil(num_classes / num_cols)

#     plt.figure(figsize=(num_cols * 3, num_rows * 3))
#     for i, class_name in enumerate(class_names, start=1):
#         class_path = os.path.join(dataset_dir, class_name)
#         image_path = os.path.join(
#             class_path, os.listdir(class_path)[0]
#         )  # Get the first image in the class
#         img = plt.imread(image_path)

#         plt.subplot(num_rows, num_cols, i)  # Adjust grid size dynamically
#         plt.imshow(img)
#         plt.title(class_name)
#         plt.axis("off")

#     plt.tight_layout()
#     plt.show()

In [ ]:
# # Visualize sample images from training set
# visualize_sample_images(TRAIN_DIR)

### Image Preprocessing

#### Normalization

In [12]:
normalize = tf.keras.layers.Rescaling(1.0 / 255)

# Apply normalization on both Training and Validation set
normalized_training_set = training_set.map(lambda x, y: (normalize(x), y))
normalized_validation_set = validation_set.map(lambda x, y: (normalize(x), y))
normalized_test_set = test_set.map(lambda x, y: (normalize(x), y))

#### Image Augmentation

In [ ]:
# Brightness adjustment
brighten = (
    tf.keras.layers.RandomBrightness(
        factor=(-0.1, 0.1),
        value_range=(0.0, 1.0),
    ),
)  # Adjust brightness by ±20%

# Contrast adjustment
add_contrast = (tf.keras.layers.RandomContrast(factor=0.5),)  # Adjust contrast by ±20%

# Rotation
rotate = (
    tf.keras.layers.RandomRotation(
        factor=0.2,
        fill_mode="constant",
        fill_value=0.0,
    ),
)  # Rotate by ±10% (36°)

# Horizontal and vertical flips
flip = (
    tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"),
)  # Flip both horizontally and vertically

# Zoom
zoom = (
    tf.keras.layers.RandomZoom(
        height_factor=(-0.2, 0.2),
        width_factor=(-0.2, 0.2),
        fill_mode="constant",
        fill_value=0.0,
    ),
)  # Zoom in/out by 20%

# Gaussian noise
add_noise = (tf.keras.layers.GaussianNoise(stddev=0.01),)  # Add Gaussian noise

In [13]:
data_augmentation = Sequential([brighten, add_contrast, rotate, flip, zoom, add_noise])

# Apply augmentation to the training set
augmented_training_set = normalized_training_set

#### Image Enhancement (not implemented)

In [14]:
# Image Enhancements have not strongly proven to increase the accuracy

#### Image Preprocessing Details

##### Augmentation Details

In [ ]:
# Test the augmentation pipeline with individual augmentations
def visualize_individual_augmentations(image_path):
    """
    Visualizes the effect of individual augmentations on an input image.

    Args:
        image_path (str): The path to the input image.

    Applies a series of individual augmentations to the input image and displays the results in a grid.
    """
    # Load and preprocess the image
    image = load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))  # Adjust to your image size
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension

    # Define individual augmentation layers
    augmentations = [
        ("Original", None),
        (
            "Random Brightness",
            brighten,
        ),
        ("Random Contrast", add_contrast),
        (
            "Random Rotation",
            rotate,
        ),
        ("Random Flip", flip),
        (
            "Random Zoom",
            zoom,
        ),
        ("Gaussian Noise", add_noise),
    ]

    # Apply each augmentation and plot
    plt.figure(figsize=(20, 5))
    for i, (title, layer) in enumerate(augmentations, start=1):
        if layer is None:
            augmented_image = image_array[0]
        else:
            augmented_image = layer(image_array)[0]

        plt.subplot(1, len(augmentations), i)
        plt.imshow(augmented_image.numpy())
        plt.title(title)
        plt.axis("off")

    plt.show()

In [ ]:
# Visualize individual augmentation techniques
visualize_individual_augmentations(SAMPLE_IMAGE)

In [ ]:
# Test the augmentation pipeline with a sample image
def visualize_augmentation(image_path):
    """
    Visualizes the effect of the augmentation pipeline on a sample image.

    Args:
        image_path (str): The path to the input image.

    Applies the augmentation pipeline to the input image and displays the original image alongside 5 augmented versions.
    """

    # Load and preprocess the image
    image = load_img(
        image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE)
    )  # Adjust to your image size
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension

    # Apply augmentations
    augmented_images = [data_augmentation(image_array)[0] for _ in range(5)]

    # Plot original and augmented images
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 6, 1)
    plt.imshow(image_array[0])
    plt.title("Original")
    plt.axis("off")

    for i, aug_img in enumerate(augmented_images, start=2):
        plt.subplot(1, 6, i)
        plt.imshow(aug_img.numpy())
        plt.title(f"Augmented {i-1}")
        plt.axis("off")
    plt.show()

In [ ]:
# Visualize the Original vs Augmented Image
visualize_augmentation(SAMPLE_IMAGE)

### Training Model

#### Building Model

In [ ]:
# Define the number of classes in your project (38 classes)
# num_classes = 72

# Load InceptionV3 model pre-trained on ImageNet without the top (classification) layer
base_model = EfficientNetV2B3(
    weights="imagenet", include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)

# Freeze the base model (don't update weights during training)
base_model.trainable = False

In [16]:
# Add custom layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)  # Reduce spatial dimensions
x = BatchNormalization()(x)  # Normalize features to improve training stability
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.4)(x)  # Dropout for regularization
x = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)  # Add a smaller dense layer for hierarchical learning
x = Activation("relu")(x)
x = Dropout(0.25)(x)  # Another dropout layer with lower rate

predictions = Dense(NUM_CLASSES, activation="softmax")(x)  # Output layer

In [17]:
# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

#### Setting Up Callbacks for Early Stopping and Model Checkpointing

In [18]:
# Define the callbacks
checkpoint = ModelCheckpoint(
    filepath=f"../models/checkpoints/{MODEL_NAME}_best_weights_{now.strftime("%Y_%m_%d_%I_%M_%S_%p")}.keras",
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    mode="max",
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=5,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
    min_lr=1e-6,
)

callbacks_list = [checkpoint, early_stopping, lr_scheduler]

#### Compiling Model

In [20]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

#### Training Model

In [ ]:
# Get the number of samples in the training and validation datasets
# train_samples = len(training_set)
# validation_samples = len(
#     validation_set
# )

# train_samples = sum([len(files) for _, _, files in os.walk(TRAIN_DIR)])
# validation_samples = sum([len(files) for _, _, files in os.walk(VALID_DIR)])

train_samples = sum(len(images) for images, _ in training_set)
validation_samples = sum(len(images) for images, _ in validation_set)

print("train_samples:", train_samples)
print("validation_samples:", validation_samples)

# Calculate steps per epoch and validation steps
steps_per_epoch = (train_samples + (BATCH_SIZE - 1)) // BATCH_SIZE
validation_steps = (validation_samples + (BATCH_SIZE - 1)) // BATCH_SIZE

print("steps_per_epoch:", steps_per_epoch)
print("validation_steps:", validation_steps)

# Compute class weights to balance the dataset
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(validation_set.class_names),
    y=validation_set.class_names,
)
class_weights_dict = dict(enumerate(class_weights))

print("Class weights:", class_weights_dict)

In [ ]:
# Train the model
training_history = model.fit(
    augmented_training_set.repeat(),
    epochs=EPOCHS,
    validation_data=normalized_validation_set.repeat(),
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weights_dict,
    callbacks=callbacks_list,
    verbose=1,
)

In [ ]:
# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Continue training with fine-tuning
fine_tuning_history = model.fit(
    augmented_training_set.repeat(),
    epochs=20,  # Additional fine-tuning epochs
    validation_data=normalized_validation_set.repeat(),
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weights_dict,
    callbacks=callbacks_list,
    verbose=1,
)

#### Evaluating Model

In [ ]:
# Training set Accuracy
train_loss, train_acc = model.evaluate(augmented_training_set)
print("Training accuracy:", train_acc)

In [ ]:
# Validation set Accuracy
val_loss, val_acc = model.evaluate(normalized_validation_set)
print("Validation accuracy:", val_acc)

#### Saving Model

In [51]:
MODEL_SAVE_DIR = (
    BASE_DIR / "models" / f"{MODEL_NAME}_{now.strftime("%Y_%m_%d_%I_%M_%S_%p")}.keras"
)

model.save(MODEL_SAVE_DIR)

In [52]:
# Save the training history
TRAIN_HIS_DIR = BASE_DIR / "training_histories" / f"training_history_{MODEL_NAME}_{now.strftime("%Y_%m_%d_%I_%M_%S_%p")}.json"
FINE_TRAIN_HIS_DIR = BASE_DIR / "training_histories" / f"fine_tunning_training_history_{MODEL_NAME}_{now.strftime("%Y_%m_%d_%I_%M_%S_%p")}.json"
with open(
    TRAIN_HIS_DIR,
    "w",
) as f:
    json.dump(training_history.history, f)

with open(
    FINE_TRAIN_HIS_DIR,
    "w",
) as f:
    json.dump(fine_tuning_history.history, f)

#### Model Details

##### Model Architecture

In [ ]:
model.summary()

##### Model Performance Metrics

In [ ]:
# Get true labels
y_true = np.concatenate([y.numpy() for _, y in test_set], axis=0)

if y_true.ndim > 1:  # If it's one-hot encoded
    y_true = np.argmax(y_true, axis=1)

print(f"y_true shape: {y_true.shape}")

In [ ]:
# Predict labels using the trained model
y_pred = model.predict(normalized_test_set)

if y_pred.ndim > 1:  # If it's one-hot encoded or probabilities
    y_pred = np.argmax(y_pred, axis=1)

print(f"y_pred shape: {y_pred.shape}")

In [ ]:
# Generate the classification report
report = classification_report(y_true, y_pred, target_names=test_set.class_names)

print("Classification Report:")
print(report)

##### Model Confusion Matrix

In [57]:
def plot_confusion_matrix_heatmap(model, test_set, class_names):
    """
    Plots the confusion matrix as a heatmap for a given model and validation dataset.
    Uses human-readable class names for display.

    Parameters:
        model: Trained model.
        test_set: Test dataset (normalized).
        class_names: List of class names.
    """
    # Get true labels and predictions
    true_labels = np.concatenate([y for x, y in test_set], axis=0)
    predicted_probs = model.predict(test_set)

    # If true_labels are one-hot encoded, convert them to class indices
    if true_labels.ndim > 1:  # Check if one-hot encoded
        true_labels = np.argmax(true_labels, axis=1)

    # Convert predicted probabilities to class indices
    predicted_labels = np.argmax(predicted_probs, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Plot confusion matrix as a heatmap
    plt.figure(figsize=(40, 40))
    sns.heatmap(
        cm,
        annot=True,
        annot_kws={"size": 10},
        cmap="magma",
        xticklabels=class_names,
        yticklabels=class_names,
    )

    plt.xlabel("Predicted Class", fontsize=20)
    plt.ylabel("Actual Class", fontsize=20)
    plt.title("Plant Disease Prediction Confusion Matrix", fontsize=25)
    plt.show()

In [ ]:
plot_confusion_matrix_heatmap(model, normalized_test_set, test_set.class_names)

##### Train | Vaild Accuracy & Loss graph

In [59]:
def plot_training_history(history):
    """
    Plots training and validation accuracy and loss graphs.

    Parameters:
        history: The History object returned by model.fit().
    """
    # Extract metrics
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label="Training Accuracy")
    plt.plot(epochs, val_acc, label="Validation Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label="Training Loss")
    plt.plot(epochs, val_loss, label="Validation Loss")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the graphs
plot_training_history(training_history)

## Conclusion